---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [121]:
import pandas as pd
import numpy as np
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [74]:
df1 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](?P<year>\d{2,4})')
df1.head(20)

,,0,1,year
,match,,,
0,0,03,25,93
1,0,6,18,85
2,0,7,8,71
3,0,9,27,75
4,0,2,6,96
5,0,7,06,79
6,0,5,18,78
7,0,10,24,89
8,0,3,7,86


In [75]:
df1['year']=df1['year'].apply(lambda y: int(y)+1900 if (len(y)==2) else int(y))
df1

,,0,1,year
,match,,,
0,0,03,25,1993
1,0,6,18,1985
2,0,7,8,1971
3,0,9,27,1975
4,0,2,6,1996
5,0,7,06,1979
6,0,5,18,1978
7,0,10,24,1989
8,0,3,7,1986


In [76]:
df2 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*) (?:\d{1,2}, )?(?P<year>\d{4})')
df2.head()

,,0,1,year
,match,,,
125,0,24,Jan,2001
126,0,10,Sep,2004
127,0,26,May,1982
128,0,28,June,2002
129,0,06,May,1972


In [77]:
df2['year']=df2['year'].apply(lambda y: int(y)+1900 if (len(y)==2) else int(y))
df2

,,0,1,year
,match,,,
125,0,24,Jan,2001
126,0,10,Sep,2004
127,0,26,May,1982
128,0,28,June,2002
129,0,06,May,1972
130,0,25,Oct,1987
131,0,14,Oct,1996
132,0,30,Nov,2007
133,0,28,June,1994


In [78]:
df3 = df.str.extractall(r' (\d{1,2})[/-](?P<year>\d{4})')
df3.head()

,,0,year
,match,,
344,0,6,2005
347,0,03,1980
351,0,8,1974
352,0,3,1986
354,0,3,1993


In [79]:
df3['year']=df3['year'].apply(lambda y: int(y)+1900 if (len(y)==2) else int(y))

In [80]:
df4 = df.str.extractall(r' (?P<year>\d{4}) ')
df4.head()

,,year
,match,
126,0,2004
127,0,1982
129,0,1972
130,0,1987
131,0,1996


In [81]:
df4['year']=df4['year'].apply(lambda y: int(y)+1900 if (len(y)==2) else int(y))

In [98]:
ser = df1['year'].append(df2['year'], ignore_index = False)
ser = ser.append(df3['year'], ignore_index = False)
ser = ser.append(df4['year'], ignore_index = False)

In [108]:
ser = pd.DataFrame(ser)

In [128]:
years = ser['year'].unique()
years.sort()

In [129]:
years

array([ 308, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980,
       1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991,
       1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016])

In [130]:
np.where(years==1989)[0][0]

19

In [132]:
sol = ser['year'].apply(lambda y: np.where(years==y)[0][0])
sol

     match
0    0        23
1    0        15
2    0         1
3    0         5
4    0        26
5    0         9
6    0         8
7    0        19
8    0        16
9    0         1
10   0        15
11   0         5
12   0        28
13   0         2
14   0        20
15   0        41
16   0        12
17   0         6
18   0        28
19   0         7
20   0        28
21   0         9
22   0        20
23   0         6
24   0        14
25   0        12
26   0        19
27   0         6
28   0         1
29   0        16
              ..
312  0        19
316  0        41
317  0         5
319  0         5
320  0        42
321  0        29
323  0         3
325  0        37
326  0        25
328  0        31
331  0        23
333  0        27
336  0         8
337  0        37
338  0        28
339  0        35
342  0         6
455  0        14
457  0        31
463  0        44
464  0        46
467  0        41
475  0        45
477  0        24
479  0        26
484  0        34
486  0         3
487

In [3]:
def date_sorter():
    
    # Your code here
    
    return # Your answer here